# Exemples d'implémentation d'algorithmes avec dask

## Nombres premiers

Voici un exemple d'implémentation d'un algorithme pour trouver les nombres premiers jusqu'à un certain nombre `n` en utilisant Dask.

In [3]:
from dask.distributed import Client
import time

client = Client()

def is_prime(x):
    for i in range(2, x // 2):
      if x % i == 0:
        return None
    return x

def launch(n):
  print(f"running for {n}")
  input = range(3, n)
  start = time.perf_counter()
  results = [x for x in input if is_prime(x) != None]
  end = time.perf_counter()
  print(f"Single threaded time: {end - start:.6f} seconds")

  start = time.perf_counter()
  futures = client.map(is_prime, input)
  results = client.gather(futures)
  end = time.perf_counter()
  print(f"Dask time: {end - start:.6f} seconds")

launch(100)
launch(10000)

running for 100
Single threaded time: 0.000030 seconds
Dask time: 0.089878 seconds
running for 10000
Single threaded time: 0.098536 seconds
Dask time: 5.291160 seconds


In [ ]:
client.dashboard_link

In [ ]:
client.close()

Encodage César de niveau 1 qui incrémente chaque lettre de l'alphabet d'une position. Par exemple, "abc" deviendrait "bcd".